In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 用于Python的Vertex AI SDK：AutoML表格化训练和预测

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/automl/automl-tabular-classification.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/automl/automl-tabular-classification.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/automl/automl-tabular-classification.ipynb">
        <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在Vertex AI Workbench中打开
    </a>
  </td>
</table>
<br/><br/><br/>

## 概述

本教程演示如何使用Vertex AI Python客户端库训练和部署用于在线预测的表格分类模型。

**注意**：在测试此SDK时，您可能会因训练、预测、存储或使用其他Google Cloud产品而产生费用。

了解更多关于[表格数据的分类](https://cloud.google.com/vertex-ai/docs/tabular-data/classification-regression/overview)。

### 目标

在本教程中，您将学习如何基于表格数据集训练和预测 AutoML 模型。或者，您可以使用 `gcloud` 命令行工具或在线 Cloud Console 来进行模型的训练和预测。

本教程使用以下 Google Cloud ML 服务和资源：

- Vertex AI
- AutoML Tabular

执行的步骤包括：

- 创建一个 Vertex AI 模型训练作业。
- 训练一个 AutoML Tabular 模型。
- 将模型资源部署到一个服务端点资源。
- 通过发送数据进行预测。
- 取消部署模型资源。

### 数据集

我们正在使用的数据集是 PetFinder 数据集，在Colab中可以本地访问。要了解更多关于这个数据集的信息，请访问https://www.kaggle.com/c/petfinder-adoption-prediction。

### 成本

本教程使用 Google Cloud 的可计费组件：

- Vertex AI
- Cloud Storage

了解 [Vertex AI 的定价](https://cloud.google.com/vertex-ai/pricing) 和 [Cloud Storage 的定价](https://cloud.google.com/storage/pricing)，并使用 [定价计算器](https://cloud.google.com/products/calculator/) 根据您的预期使用情况生成成本估算。

## 安装

安装执行此笔记本所需的软件包。

In [ ]:
# Install the packages
! pip3 install --quiet --upgrade google-cloud-aiplatform \
                                 google-cloud-storage

仅限Colab：取消下面的单元格注释以重新启动内核

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

## 在开始之前

### 设置您的项目ID

**如果您不知道您的项目ID**，请尝试以下操作：
* 运行 `gcloud config list`。
* 运行 `gcloud projects list`。
* 查看支持页面：[找到项目ID](https://support.google.com/googleapi/answer/7014113)

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

#### 区域

您还可以更改 Vertex AI 使用的 `REGION` 变量。了解更多关于 [Vertex AI 区域](https://cloud.google.com/vertex-ai/docs/general/locations)。

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

### 认证您的 Google Cloud 帐户

根据您的 Jupyter 环境，您可能需要手动进行身份验证。请按照以下相关说明进行操作。

**1. Vertex AI Workbench**
* 不需要进行任何操作，因为您已经通过身份验证。

**2. 本地 JupyterLab 实例，请取消注释并运行：**

In [ ]:
# ! gcloud auth login

3. 合作，取消注释并运行：

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

4. 服务账号或其他
* 查看如何在https://cloud.google.com/storage/docs/gsutil/commands/iam#ch-examples上向您的服务账号授予Cloud Storage权限。

创建一个云存储桶
创建一个存储桶，用于存储中间产物，比如数据集。

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

如果不存在您的存储桶：运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_URI

将数据集复制到您的云存储存储桶中###

In [ ]:
IMPORT_FILE = "petfinder-tabular-classification.csv"
! gsutil cp gs://cloud-samples-data/ai-platform-unified/datasets/tabular/{IMPORT_FILE} {BUCKET_URI}/data/

gcs_source = f"{BUCKET_URI}/data/{IMPORT_FILE}"

### 导入用于 Python 的 Vertex AI SDK

将 Vertex AI SDK 导入到 Python 环境中并进行初始化。

In [ ]:
import os

from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=REGION)

## 教程

现在您已经准备好创建您的AutoML表格模型了。

### 从CSV文件创建托管的表格数据集

这一部分将从存储在您的GCS存储桶中的CSV文件创建一个数据集。

In [ ]:
ds = dataset = aiplatform.TabularDataset.create(
    display_name="petfinder-tabular-dataset",
    gcs_source=gcs_source,
)

ds.resource_name

启动一个训练作业以创建一个模型

一旦你定义了训练脚本，你将创建一个模型。`run` 函数创建一个训练管道，用于训练和创建一个模型对象。训练管道完成后，`run` 函数会返回模型对象。

In [ ]:
job = aiplatform.AutoMLTabularTrainingJob(
    display_name="train-petfinder-automl-1",
    optimization_prediction_type="classification",
    column_transformations=[
        {"categorical": {"column_name": "Type"}},
        {"numeric": {"column_name": "Age"}},
        {"categorical": {"column_name": "Breed1"}},
        {"categorical": {"column_name": "Color1"}},
        {"categorical": {"column_name": "Color2"}},
        {"categorical": {"column_name": "MaturitySize"}},
        {"categorical": {"column_name": "FurLength"}},
        {"categorical": {"column_name": "Vaccinated"}},
        {"categorical": {"column_name": "Sterilized"}},
        {"categorical": {"column_name": "Health"}},
        {"numeric": {"column_name": "Fee"}},
        {"numeric": {"column_name": "PhotoAmt"}},
    ],
)

# This takes about an hour to run
model = job.run(
    dataset=ds,
    target_column="Adopted",
    training_fraction_split=0.8,
    validation_fraction_split=0.1,
    test_fraction_split=0.1,
    model_display_name="adopted-prediction-model",
    disable_early_stopping=False,
)

### 部署您的模型

在使用您的模型进行预测之前，您需要将其部署到一个端点。您可以通过在模型资源上调用`deploy`函数来实现这一点。此函数会执行两个操作：

1. 创建一个端点资源，将模型资源部署到该端点。
2. 将模型资源部署到端点资源上。

部署您的模型。

### 注意：在继续预测之前，请等待模型**完成**部署。

In [ ]:
endpoint = model.deploy(
    machine_type="n1-standard-4",
)

### 预测在终点

这个样本实例取自一个观察，其中`Adopted` = **是**
请注意，所有的值都是字符串。由于原始数据是以CSV格式的，所以一切都被视为字符串。在创建`AutoMLTabularTrainingJob`时定义的转换规则会告诉Vertex AI将输入转换为其定义的类型。

In [ ]:
prediction = endpoint.predict(
    [
        {
            "Type": "Cat",
            "Age": "3",
            "Breed1": "Tabby",
            "Gender": "Male",
            "Color1": "Black",
            "Color2": "White",
            "MaturitySize": "Small",
            "FurLength": "Short",
            "Vaccinated": "No",
            "Sterilized": "No",
            "Health": "Healthy",
            "Fee": "100",
            "PhotoAmt": "2",
        }
    ]
)

print(prediction)

### 下线模型

要从服务端点资源中取消部署模型资源，请使用端点的 `undeploy` 方法，并提供以下参数：

- `deployed_model_id`：当模型资源部署时由预测服务返回的模型部署标识符。您可以使用预测对象的 `deployed_model_id` 属性检索 `deployed_model_id`。

In [ ]:
endpoint.undeploy(deployed_model_id=prediction.deployed_model_id)

清理

要清理此项目中使用的所有Google Cloud资源，您可以删除用于教程的[Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在本教程中创建的各个资源：

- 训练工作
- 模型
- 端点
- 云存储桶

**注意**: 在删除端点资源之前，您必须删除部署到端点资源的任何模型资源。

In [ ]:
# Warning: Setting this to true will delete everything in your bucket
delete_bucket = False

# Delete the training job
job.delete()

# Delete the model
model.delete()

# Delete the endpoint
endpoint.delete()

if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil -m rm -r $BUCKET_URI